In [1]:
from pylab import *
import cv2
import pickle
from glob import glob

opencv_home='/Users/yoavfreund/projects/opencv'
frontal=opencv_home+'/data/haarcascades/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(frontal)

## Read videos and detect faces
videos_dir="/Users/yoavfreund/projects/facesAnalysis/videos/"
output_dir="/Users/yoavfreund/projects/facesAnalysis/output/"
#filename = "Bernie Can\'t Be The Only Candidate With An Ice Cream-z3qN-BU1b2M"
#filename = "A Special Halloween Warning-mkuyOVckB8s"
filename = "Stephen's Wish List for Hillary Clinton-qUBWEMEPxDw"

print "reading and detecting faces"
vid = cv2.VideoCapture(videos_dir+filename+".mp4")
i=0;
Faces_List=[]
while True:
    flag,frame=vid.read()
    if not flag:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(100, 100),
        flags = cv2.cv.CV_HAAR_SCALE_IMAGE
    )
    Faces_List.append(faces)
    #print 'index=',i,'faces=',faces
    i+=1
    if i % 10 == 0:
        print '\r',i,

pickle.dump(Faces_List,open(output_dir+filename+'_Faces.pkl','w'),protocol=2);

print 'Computing statistics'
# collect all face boxes in frames where there is only one box
one_square = np.array([[i]+list(Faces_List[i][0])\
                       for i in range(len(Faces_List))\
                       if len(Faces_List[i])==1])

# Compute mean and std for each of the box parameters
# and for each frame-to-frame change in the parameter

mean_loc = mean(one_square,axis=0)[1:]
std_loc = std(one_square,axis=0)[1:]
mean_loc,std_loc

diffs = one_square[1:,:]-one_square[:-1,:]
consec=diffs[diffs[:,0]==1,:]

mean_diff=mean(consec,axis=0)[1:]
std_diff=std(consec,axis=0)[1:]
mean_diff,std_diff, mean_loc,std_loc

def check_face(face):
    Z=0;
    for p in range(4):
        Z+=((face[p]-mean_loc[p])/(sqrt(2)*std_loc[p]))**2
    return Z

def check_shift(face,prev_face):
    Z=0
    for p in range(4):
        Z+=((face[p]-prev_face[p])/(sqrt(2)*std_diff[p]))**2
    return Z
def find_first_one(list):
    for i in range(len(list)):
        if list[i]==1:
            return i
    return -1  # return -1 if did not find a 1 in list

Z_threhold=10.0
def find_match_with_current(current,Range=5):
    # Look for matches with box in current frame
    for this_box in Faces_List[current]:
        if check_face(this_box)>Z_threhold:
            continue
        for step in range(Range,1,-1):
            for Delta in [step,-step]:
                i=current+Delta
                if i<0 or i>=len(Faces_List):
                    continue
                for that_box in Faces_List[i]:
                    if check_face(that_box)>Z_threhold:
                        continue
                    Z=check_shift(this_box,that_box)
                    if Z<1:
                        return (this_box,i,that_box,Z)
    return None
                    
def find_match_btwn_neighbors(current,Range=5):
    # look for matches between frames before and after
    for gap in range(2*Range,2,-1):
        for left in range(current-gap+1,current):
            right=left+gap
            if left<0 or right>=len(Faces_List):
                continue
            for left_box in Faces_List[left]:
                if check_face(left_box)>Z_threhold:
                    continue
                for right_box in Faces_List[right]:
                    if check_face(right_box)>Z_threhold:
                        continue
                    Z=check_shift(left_box,right_box)
                    if Z<2:
                        return (left_box,left,right_box,right,Z)

print "cleaning detected faces to keep at most one box per frame"
Cleaned_Faces_List=[]
for current in range(len(Faces_List)): 
    match=find_match_with_current(current)
    if not match is None:
        #print current,'withFrame',match[0],check_face(match[0])
        Cleaned_Faces_List.append((current,match[0]))
    else:
        match = find_match_btwn_neighbors(current)
        if match is None:
            #print 'No Match'
            Cleaned_Faces_List.append((current,None))
        else:
            left_box,left_i,right_box,right_i,Z=match
            left_weight=right_i-current
            right_weight=current - left_i
            box=((left_weight*left_box)+(right_weight*right_box))/(left_weight+right_weight)
            box=box.astype(int)
            #print current,'btwn neighbors',box,check_face(box)
            Cleaned_Faces_List.append((current,box))
    if current % 100 == 0:
        print '\r',current,

size=300
def extract_face(i,image):
    if Cleaned_Faces_List[i][1] is None:
        return None
    image_h,image_w,c=shape(image)
    x,y,w,h=list(Cleaned_Faces_List[i][1])
    y0=y+h/2
    y0=min(image_h-size/2,max(size/2,y0))
    x0=x+w/2
    x0=min(image_w-size/2,max(size/2,x0))
    ymin=y0-size/2; ymax=ymin+size
    xmin=x0-size/2; xmax=xmin+size
    return (i,x0,y0,w,h,image[ymin:ymax,xmin:xmax,:])

print "Extracting windows around detected faces"
fourcc = fourcc = cv2.cv.CV_FOURCC(*'mp4v') #cv2.VideoWriter_fourcc('mp4v')
out = cv2.VideoWriter(output_dir+filename+'_Windows.mp4',fourcc, 20.0, (size,size))

vid = cv2.VideoCapture(videos_dir+filename+".mp4")
Windows=[]
for j in range(len(Cleaned_Faces_List)):
    flag,image = vid.read()
    if not flag:
        print 'got to video file EOF'
        break
    I=extract_face(j,image)
    if not I is None:
        #print j,I[:-1]
        out.write(I[-1])
        Windows.append(I)
    if j % 10 == 0:
        print '\r',j,
    #imshow_rgb(I,'window '+str(i))
out.release()

pickle.dump(Windows,open(output_dir+filename+'_Windows.pkl','w'),protocol=2);

print 'Done'

reading and detecting faces
1810 Computing statistics
cleaning detected faces to keep at most one box per frame
1800 Extracting windows around detected faces
1810 Done


In [ ]:
!ls -lrt ../output/

In [17]:
len(Windows)

77

In [18]:
Cleaned_Faces_List[60:70]

[(60, None),
 (61, None),
 (62, None),
 (63, None),
 (64, array([560,  84, 150, 150], dtype=int32)),
 (65, array([560,  86, 148, 148], dtype=int32)),
 (66, array([561,  86, 148, 148], dtype=int32)),
 (67, array([562,  88, 147, 147], dtype=int32)),
 (68, array([564,  89, 148, 148], dtype=int32)),
 (69, array([567,  89, 148, 148], dtype=int32))]

In [25]:
!ls -l ../videos/

total 19820824
-rw-r--r--  1 yoavfreund  staff   65123489 Nov  7  2015 'Homeland' Suffered A Major Intelligence Failure-9IE1mUL1erk.mp4
-rw-r--r--  1 yoavfreund  staff   52786819 Jan 15 23:40 19 Days And Counting-JxQKYgw9G2k.mp4
-rw-r--r--  1 yoavfreund  staff   52870961 Jan 20 03:37 A Modest PromPosal For Bernie Sanders-kRsRUIPoW4E.mp4
-rw-r--r--  1 yoavfreund  staff   24235611 Oct  7  2015 A Moment For South Carolina-GDFrVwgicsc.mp4
-rw-r--r--  1 yoavfreund  staff   80281294 Nov  1  2015 A Special Halloween Warning-mkuyOVckB8s.mp4
-rw-r--r--  1 yoavfreund  staff   74703758 Dec 11 06:57 A Tahini Bit Of Perspective On Politics-q-cirwvF-SI.mp4
-rw-r--r--  1 yoavfreund  staff   36090145 Jan 20 20:05 A Tribute To Glenn Frey-0DaJGhJz7SY.mp4
-rw-r--r--@ 1 yoavfreund  staff   85792248 Mar 16 21:41 All You Can Trump Buffet-F5zjVUZA7rY.mp4
-rw-r--r--  1 yoavfreund  staff   15796063 Feb 21 04:44 And Now, Some Totally Organic Product Placement, Part 2-ua1jbS-je8I.mp4
-rw-r--r--  1 yoavfreund  st

In [16]:
%%writefile detect_faces.py
from pylab import *
import cv2
import pickle
from glob import glob

opencv_home='/Users/yoavfreund/projects/opencv'
frontal=opencv_home+'/data/haarcascades/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(frontal)

## Read videos and detect faces
videos_dir="/Users/yoavfreund/projects/facesAnalysis/videos/"
output_dir="/Users/yoavfreund/projects/facesAnalysis/output/"

for path in glob(videos_dir+"/*"):
    start=path.find('videos/')+7
    filename=path[start:-4]
    print filename
    print "reading and detecting faces"
    vid = cv2.VideoCapture(videos_dir+filename+".mp4")
    i=0;
    Faces_List=[]
    while True:
        flag,frame=vid.read()
        if not flag:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the image
        faces = faceCascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(100, 100),
            flags = cv2.cv.CV_HAAR_SCALE_IMAGE
        )
        Faces_List.append(faces)
        #print 'index=',i,'faces=',faces
        i+=1
        if i % 10 == 0:
            print '\r',i,

    pickle.dump(Faces_List,open(output_dir+filename+'_Faces.pkl','w'),protocol=2);


Writing detect_faces.py


In [41]:
%%writefile detect_faces_aws.py
from pylab import *
import cv2
import pickle
from glob import glob
import sys

start=int(sys.argv[1])
step=int(sys.argv[2])

print start,step

videos_dir="/run/shm/videos/"
file_list=glob(videos_dir+'*')

sub_list=[file_list[i] for i in range(start,len(file_list),step)]
print 'files to be processed:'
print '\n'.join(sub_list)
print

frontal='/usr/share/opencv/haarcascades/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(frontal)

## Read videos and detect faces
output_dir="/run/shm/output/"

for path in sub_list:
    start=path.find('videos/')+7
    filename=path[start:-4]
    print filename
    print "reading and detecting faces"
    vid = cv2.VideoCapture(videos_dir+filename+".mp4")
    i=0;
    Faces_List=[]
    while True:
        flag,frame=vid.read()
        if not flag:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the image
        faces = faceCascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(100, 100),
            flags = cv2.cv.CV_HAAR_SCALE_IMAGE
        )
        Faces_List.append(faces)
        #print 'index=',i,'faces=',faces
        i+=1
        if i % 10 == 0:
            print '\r',i,

    pickle.dump(Faces_List,open(output_dir+filename+'_Faces.pkl','w'),protocol=2);

Overwriting detect_faces_aws.py


In [14]:
filename[48:-4]

"'Homeland' Suffered A Major Intelligence Failure-9IE1mUL1erk"